In [8]:
# EXPLORATION DATA ANALYSIS & DATASET CREATE
import pandas as pd
import numpy as np
import os

In [3]:
def label2idx(result):
    results = result.copy()
    for i, result in enumerate(results):
        results[i] = int(result.split('/')[-1][1:])

    return results


def get_fact(content):
    breakers = []
    fact_starts = []

    for i, c in enumerate(content):
        if not c : continue
        if c[0] == '【' and c[-1] == '】':
            breakers.append(i)
            if '查明' in c:
                fact_starts.append(i)

    fact = []
    
    for start in fact_starts:
        end = breakers[breakers.index(start)+1]
        fact += content[start:end]
    
    return ''.join(fact)


def get_level3labels(tree):
    level3labels = []
    for t in tree:
        level1 = t['value']
        children1 = t['children']
        for child1 in children1:
            level2 = child1['value']
            children2 = child1['children']
            for child2 in children2:
                level3 = child2['value']
                level3labels.append('/'.join([level1, level2, level3]))
    return level3labels

In [7]:
import json
import torch
from torch.utils.data import Dataset

class CaseData(Dataset):
    def __init__(self, file_name, class_num,is_debug):
        self.data = json.load(open(file_name, encoding='utf-8'))
        if is_debug: self.data = self.data[0:200]
        self.class_num = class_num

    def __getitem__(self, idx):
        fact = get_fact(self.data[idx]['content'])
        label = torch.zeros(self.class_num)
        for i in label2idx(self.data[idx]['result']):
            label[i] = 1

        return fact, label

    def __len__(self):
        return len(self.data)


In [9]:
code_tree=json.load(open('../rawdata/code_tree.json', encoding='utf-8'))
textual_tree = json.load(open('../rawdata/textual_tree.json', encoding='utf-8'))
data=json.load(open('../rawdata/train.json', encoding='utf-8'))

textual_table = []
code_table = []

def tree_to_table(tree):
    table= []
    for i in tree:
        a = i["value"]
        for j in i["children"]:
            b = j["value"]
            for z in j["children"]:
                c=z["value"]
                table.append([a,b,c])
    return table

textual_table = tree_to_table(textual_tree)
code_table = tree_to_table(code_tree)
textual_frame = pd.DataFrame(textual_table,columns=["A","B","C"])
code_frame = pd.DataFrame(code_table,columns=["A","B","C"])

text_code_frame = pd.concat([code_frame["C"],textual_frame["C"]],axis=1)
text_code_frame.columns = ["code_label","textual_label"]

code_dict = dict(zip(code_frame["C"],textual_frame["C"]))
print("code_dict size:",len(code_dict))
print("code_frame size",code_frame.shape[0])
print("text_frame size",textual_frame.shape[0])


process_data_path = '../preprocessdata/'
if not os.path.exists(process_data_path):
    os.mkdir(process_data_path)
textual_frame.to_csv(process_data_path+'textual_frame.csv',index=False)
code_frame.to_csv(process_data_path+'code_frame.csv',index=False)
text_code_frame.to_csv(process_data_path+'text_code_frame.csv',index=False)

code_dict size: 234
code_frame size 234
text_frame size 234


In [4]:
# result number 和 evidence number的个数是一样的，而且都小于给定的标签。
data=json.load(open('../rawdata/train.json', encoding='utf-8'))
result_set = set()
for i in data:
    for j in i["result"]:
        result_set.add(j)
print("result_number",len(result_set))
evidence_set = set()
for i in data:
    for j in i["evidence"]:
        for z in j["value"]:
            evidence_set.add(z)
print("evidence_number",len(result_set))

result_number 216
evidence_number 216


In [13]:

case = data[0]
evidence = case["evidence"]
for i in evidence:
    idx = i["index"]
    text = ''
    for j in i["value"]:
        code_label=j.split('/')[-1]
        text_label= code_dict[code_label]
        text+=text_label+'/'

    print('标签:',text,'\n','文本:',case["content"][idx])
    print('上文：',case["content"][idx-1],'\n下文：',case["content"][idx+1],'\n','*'*20,'\n')
    

标签: 自认/ 
 文本: 温玉富分别于2012年7月6日、2012年12月16日、2013年1月31日、2013年2月1日、2013年3月11日向刘明转账汇款100万元、150万元、30万元、20万元、100万元，刘明认可收到了上述400万元。
上文： 刘明主张其为鑫丰汇川公司的实际控制人。 
下文： 温玉富主张其曾于2010年以转账支票的方式向刘明出借50万元，支票收款人为鑫丰汇川公司，收款时间为2010年10月12日，该笔款项于2010年10月13日自鑫丰汇川公司的浦发银行账户转出； 
 ******************** 

标签: 票据交付借款/ 
 文本: 温玉富主张其曾于2010年以转账支票的方式向刘明出借50万元，支票收款人为鑫丰汇川公司，收款时间为2010年10月12日，该笔款项于2010年10月13日自鑫丰汇川公司的浦发银行账户转出；
上文： 温玉富分别于2012年7月6日、2012年12月16日、2013年1月31日、2013年2月1日、2013年3月11日向刘明转账汇款100万元、150万元、30万元、20万元、100万元，刘明认可收到了上述400万元。 
下文： 刘明在（2015）怀民（商）初字第03921号案中认可收到了该笔款项，但在本案中予以否认。 
 ******************** 

标签: 自认/ 
 文本: 刘明在（2015）怀民（商）初字第03921号案中认可收到了该笔款项，但在本案中予以否认。
上文： 温玉富主张其曾于2010年以转账支票的方式向刘明出借50万元，支票收款人为鑫丰汇川公司，收款时间为2010年10月12日，该笔款项于2010年10月13日自鑫丰汇川公司的浦发银行账户转出； 
下文： 温玉富主张其曾受泛珠泉公司委托向案外人汪某某、赵某某借款，且泛珠泉公司曾对此予以确认。 
 ******************** 

标签: 委托代理人/ 
 文本: 温玉富主张其曾受泛珠泉公司委托向案外人汪某某、赵某某借款，且泛珠泉公司曾对此予以确认。
上文： 刘明在（2015）怀民（商）初字第03921号案中认可收到了该笔款项，但在本案中予以否认。 
下文： 温玉富提举了泛珠泉公司出具的借款证明，其与赵某某签订的借款合同和补充协议，以及其与汪某某签订的借款合同。 
 ********************

In [8]:
content_list = []
content_list.append([1,2,3])
content_list.append([1,2,3])
content_list.append(content_list)
content_list

[[1, 2, 3], [1, 2, 3], [...]]

In [12]:
# def create_dataset()
from tqdm import tqdm
columns = ['text','label','content']
label_size=textual_frame.shape[0]
process_data = []
for i in tqdm(data):
    content_list = []
    for j in i["content"]:
        content_list.append([i["id"],label_size,j])
    for j in i["evidence"]:
        idx = j["index"]
        value_list = j["value"]
        pro_value_list = []
        for z in value_list:
            code_label = z.split('/')[-1]
            num_label = code_frame[code_frame["C"]==code_label].index
            pro_value_list.append(str(num_label))
               
        content_list[idx] = '#'.join(pro_value_list)
    if not process_data:
        process_data = content_list
    else:
        process_data=process_data+content_list
print(process_data)
# dataset = pd.DataFrame(process_data,columns=columns)
# dataset.head(5)

100%|██████████| 2496/2496 [00:12<00:00, 205.69it/s]

In [ ]:
import numpy as np
vectorlabel_list = []
vectorlabel = np.zeros(4)
vectorlabel_list.append(vectorlabel)
vectorlabel_list.append(vectorlabel)


array([[0., 0., 0., 0.],
       [0., 0., 0., 0.]])